In [1]:
import numpy as np
import scipy
import csv
import matplotlib.pyplot as plt
from py3gpp import *

In [2]:
input_file = '/home/tiwat/workarea/IQ_constellation/iq_python/input/IQDataFile_SPEEDTEST_10MS.csv'
# input_file = '/home/tiwat/workarea/IQ_constellation/iq_python/input/IQDataFile_SPOTIFY2.csv'

with open(input_file, 'r') as f:
    reader = csv.reader(f)
    for _ in range(19):
        next(reader)

    iq_val = []
    for row in reader:
        i_val = float(row[0])
        q_val = float(row[1])
        iq_val.append(complex(i_val,q_val))     # I+jQ

    waveform_csv = np.array(iq_val[:230401])    # 10 ms
    # waveform_csv = np.array(iq_val)

print("len(waveform)", len(waveform_csv))
print("sample waveform_csv")
waveform_csv[:5]

len(waveform) 230401
sample waveform_csv


array([-0.00123117-0.00102036j,  0.00109794-0.00055222j,
        0.00108245-0.00025547j,  0.00074913-0.00107994j,
        0.00054957-0.00111326j])

In [3]:
# downsampling, low pass filter
dec_factor = 8
waveform = scipy.signal.decimate(waveform_csv, dec_factor, ftype='fir')
print("len(waveform)",len(waveform))
print("waveform after decimate:",waveform[:5])

# scale up max amplitude to 1
waveform /= max(waveform.real.max(), waveform.imag.max())
print("waveform scale max amplitude to 1:",waveform[:5])

samplingRate = 23.04e6
fs = samplingRate // dec_factor     # floor devision   

len(waveform) 28801
waveform after decimate: [ 0.00039785-0.00057816j  0.00049174-0.00041159j -0.00012423+0.00074921j
 -0.00019332+0.00036641j -0.00030398+0.00052994j]
waveform scale max amplitude to 1: [ 0.16179951-0.23512766j  0.19998263-0.167384j   -0.05051995+0.30468921j
 -0.07861919+0.1490128j  -0.12362406+0.21551413j]


In [31]:
t = np.arange((len(waveform)/fs)*1e3)   # 10 ms

In [26]:
scs = 30
syncNfft = 256
syncSR = int(syncNfft * scs * 1e3)
scipy.signal.resample(waveform, syncSR)

array([0.16179951-0.23512766j, 0.16211793-0.23636639j,
       0.16243185-0.23760308j, ..., 0.16081647-0.23140063j,
       0.16114887-0.23264464j, 0.16147652-0.23388703j])

In [117]:
carrier = nrCarrierConfig(NSizeGrid = 20, SubcarrierSpacing = scs)
info = nrOFDMInfo(carrier)
Nfft = info['Nfft']

pssIndices = np.arange((119-63), (119+64))
NID2 = 2
refGrid = np.zeros((20*12,2))
refGrid[pssIndices,1] = nrPSS(NID2)
print('refGrid:',refGrid[1], refGrid[100])
nslot = 0

slotGrid = nrResourceGrid(carrier)
slotGrid = slotGrid[:, 0]
slotGrid[pssIndices] = nrPSS(NID2)
print(slotGrid[1], slotGrid[56])



refGrid: [0. 0.] [0. 1.]


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed